In [260]:
import torch
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"{torch.__version__} running on {device}")

2.4.0+cu121 running on cuda


# GP Run for Image Classification
Your problem needs to fulfill the following criteria.
1. It is an image classification problem.
2. You supply marked training images and marked validation images.

Within those, the run is flexible and adapts itself to your problem.
Now, please describe your images and problem by setting those global variables:

In [261]:
IMAGE_WIDTH = 28 # <-- number of width-pixels
IMAGE_HEIGHT = 28 # <-- number of height-pixels
COLOUR_CHANNEL_COUNT = 1 # <-- RGB images would have 3
CLASSIFICATION_CATEGORIES_COUNT = 10 # <-- the amount of possible categories of which each image shall be marked with one

## The Individuals' Class
- The Hyperparamters should be passed to the constructor in a way that is both convenient for GP and for PyTorch.
- I think I want to define a class for one `nn.Sequential` 2d-block
    - All possible instances should be concatenable with all possible instances
- Then, an individual is built from the concatenation of many such blocks, plus data preparation and final f.c. layer

In [262]:
''' 
define a λ nn.Module that creates an nn layer from a given function
this is handy for nn.Sequential usage '''
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)
    
'''
create a function to reshape the (28x28) image input data '''
def preprocess(x):
    return x.view(-1, 1, 28, 28)

In [263]:
class Sequential_block_2d(nn.Module):
    def __init__(self, 
                 out_channels: int, # the number of output neurons after the full block
                 in_channels: int = 1, # should not be set here, but is set in Individual's __init__()
                 conv_kernel_size: int = 3,
                 conv_stride: int = 1,
                 conv_padding: int = 1,
                 pool_kernel_size: int = 2,
                 pool_stride: int = 2,
                 pool_padding: int = 0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=conv_kernel_size,
                      stride=conv_stride,
                      padding=conv_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_kernel_size,
                         stride=pool_stride,
                         padding=pool_padding)
        )
    def forward(self, x):
        return self.block(x)

# generate some testing blocks (TODO: write actual unit tests)
testBlock1 = Sequential_block_2d(in_channels=1,out_channels=5)
testBlock2 = Sequential_block_2d(in_channels=5,out_channels=3)
torch.manual_seed(42)
testX = torch.randn(COLOUR_CHANNEL_COUNT,IMAGE_WIDTH,IMAGE_HEIGHT)
testBlock1, testBlock2, testX, testBlock1(testX)
print(f"testX.shape = {testX.shape}\ntestX[:,:3]: {testX[:,:3]}")
print(f"testBlock1(testX).shape = {testBlock1(testX).shape}\ntestBlock1(testX)[:1,:3]: {testBlock1(testX)[:1,:3]}")

testX.shape = torch.Size([1, 28, 28])
testX[:,:3]: tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431,
          -1.6047, -0.7521,  1.6487, -0.3925, -1.4036, -0.7279, -0.5594,
          -0.7688,  0.7624,  1.6423, -0.1596, -0.4974,  0.4396, -0.7581,
           1.0783,  0.8008,  1.6806,  1.2791,  1.2964,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599, -1.3847, -0.8712, -0.2234,
           1.7174,  0.3189, -0.4245,  0.3057, -0.7746, -1.5576,  0.9956,
          -0.8798, -0.6011, -1.2742,  2.1228, -1.2347, -0.4879, -0.9138,
          -0.6581,  0.0780,  0.5258, -0.4880,  1.1914, -0.8140, -0.7360],
         [-1.4032,  0.0360, -0.0635,  0.6756, -0.0978,  1.8446, -1.1845,
           1.3835,  1.4451,  0.8564,  2.2181,  0.5232,  0.3466, -0.1973,
          -1.0546,  1.2780, -0.1722,  0.5238,  0.0566,  0.4263,  0.5750,
          -0.6417, -2.2064, -0.7508,  0.0109, -0.3387, -1.3407, -0.5854]]])
testBlock1(testX).shape = torch.Size([5, 14, 14])
testBlock1(testX)[

In [264]:
# class for image classification individuals
class NN_individual(nn.Module):
    def __init__(self, blocks_2d: list[Sequential_block_2d],
                 in_dimensions: int = COLOUR_CHANNEL_COUNT,
                 out_dimensions: int = CLASSIFICATION_CATEGORIES_COUNT,
                 fin_res: int = 5): # output dimension of the final max pooling 
                                    # producing size (fin_res * fin_res)
        super().__init__()
        self.blocks_2d = blocks_2d
        # add a final max pool
        # Why? Because then torch handles the dimensions through the "adaptive"ness
        self.last_max_pool_adaptive = nn.AdaptiveAvgPool2d((fin_res, fin_res))
        self.flatten = nn.Flatten(start_dim=0, end_dim=-1) # default start_dim = 1
        self.lin = nn.Linear(in_features = fin_res * fin_res * blocks_2d[-1].out_channels,
                      out_features = out_dimensions)
    def forward(self, x):
        for i in range(len(self.blocks_2d)):
            x = self.blocks_2d[i](x)
        x = self.last_max_pool_adaptive(x)
        #print(f"last_max_pool_adaptive output shape is {x.shape}")
        x = self.flatten(x)
        #print(f"flatten output shape is {x.shape}")
        x = self.lin(x)
        #print(f"lin output shape is {x.shape}")
        return x
    
testIndividual = NN_individual(blocks_2d=[testBlock1, testBlock2])
testIndividual(testX)

tensor([-0.0964, -0.0073, -0.2285, -0.0881, -0.0110,  0.2158,  0.0744,  0.2094,
        -0.1230,  0.4733], grad_fn=<ViewBackward0>)

In [265]:
# this way, adjacent genes (= 2d_blocks) need to have the correct dimensions
# e.g. the following will error:
badIndividual = NN_individual([Sequential_block_2d(in_channels=1,out_channels=2), Sequential_block_2d(in_channels=3, out_channels=5)])
try:
    print(badIndividual(testX))
except:
    print("out_channels of the first needs to match in_channels of the second!")

# but there's more redundancy:\the first gene needs to have the same number of in_channels as there are colour channels
# e.g. the following will error:
badIndividual = NN_individual([Sequential_block_2d(in_channels=COLOUR_CHANNEL_COUNT + 1,out_channels=5)])
try:
    print(badIndividual(testX))
except:
    print("in_channels of the first gene needs to match the COLOUR_CHANNEL_COUNT of the problem!")

out_channels of the first needs to match in_channels of the second!
in_channels of the first gene needs to match the COLOUR_CHANNEL_COUNT of the problem!


### Brainstorm on How to Encode Individuals
We need to talk about this right now because we want to adapt our `NN_individual.blocks_2d` definition according to it.
The options are:
1. We specify `Sequential_block_2d.in_channels` and `~.out_channels` separately for each individual and only allow concatenation if the criteria are met. This is probably not super clever...
2. Genotype-closure: The parameters that are adapted through GP will never leave the space of syntacticly correct indivuals
    - The first gene is not allowed to choose `~.in_channels`, it must match `COLOUR_CHANNEL_COUNT`
    - Every gene but the first is not allowed to choose `~.in_channels`, it must match `~.out_channels` of the prior gene
    - How will this change the gene class `Sequential_block_2d`?
        - Set `Sequential_block_2d.in_channels` only programatically, in `NN_individual.__init__()`
        - Don't even let the genes inherit from `nn.Module`, only the individual

In [266]:
# class for the genetic information of one 2d block
class Gene_2d_block:
    def __init__(self,
                 out_channels: int,
                 in_channels: int = None, # set in the individual's constructor
                 conv_kernel_size: int = 3,
                 conv_stride: int = 1,
                 conv_padding: int = 1,
                 pool_kernel_size: int = 2,
                 pool_stride: int = 2,
                 pool_padding: int = 0):
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.conv_kernel_size = conv_kernel_size
        self.conv_stride = conv_stride
        self.conv_padding = conv_padding
        self.pool_kernel_size = pool_kernel_size
        self.pool_stride = pool_stride
        self.pool_padding = pool_padding

    def toString(self, tab_count: int = 0):
        indentation = ""
        for tab in range(tab_count): indentation += f"\t"
        return f"{indentation}out_channels = {self.out_channels}\n"+\
        f"{indentation}conv_2d (kernel, stride, padding) =\t({self.conv_kernel_size}, {self.conv_stride}, {self.conv_padding})\n"+\
        f"{indentation}max_pool_2d (kernel, stride, padding) =\t({self.pool_kernel_size}, {self.pool_stride}, {self.pool_padding})"

In [267]:
# class for image classification individuals
class NN_individual(nn.Module):
    def __init__(self, genes_2d_block: list[Gene_2d_block], name="nn0"): 
        super().__init__()
        self.name=name # a name for easier tracking inside a GP run
        ''' build the full sequential from the gene information (genes_2d_block) '''
        self.blocks_2d = nn.Sequential()
        # the first 2d_block needs to have as many in_channels as there are colour channels
        # the others need to have as in_channels the number of out_channels from the previous block
        for i in range(len(genes_2d_block)):
            if i == 0:
                in_channels = COLOUR_CHANNEL_COUNT
            else:
                in_channels = genes_2d_block[i-1].out_channels
            self.blocks_2d.append(nn.Sequential(
                nn.Conv2d(in_channels=in_channels,
                    out_channels=genes_2d_block[i].out_channels,
                    kernel_size=genes_2d_block[i].conv_kernel_size,
                    stride=genes_2d_block[i].conv_stride,
                    padding=genes_2d_block[i].conv_padding),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=genes_2d_block[i].pool_kernel_size,
                    stride=genes_2d_block[i].pool_stride,
                    padding=genes_2d_block[i].pool_padding)))
        self.flatten = nn.Flatten(start_dim=0, end_dim=-1) # default start_dim = 1
        self.lazyLin = nn.LazyLinear(out_features = CLASSIFICATION_CATEGORIES_COUNT) # automatically infers the number of channels
    def forward(self, x):
        for i in range(len(self.blocks_2d)):
            x = self.blocks_2d[i](x)
        x = self.flatten(x)
        #print(f"flatten output shape is {x.shape}")
        x = self.lazyLin(x)
        #print(f"lin output shape is {x.shape}")
        return x
    
testIndividual = NN_individual(genes_2d_block=[Gene_2d_block(out_channels=4), Gene_2d_block(out_channels=7)])
testIndividual, testIndividual(testX)

(NN_individual(
   (blocks_2d): Sequential(
     (0): Sequential(
       (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): ReLU()
       (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (1): Sequential(
       (0): Conv2d(4, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): ReLU()
       (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (flatten): Flatten(start_dim=0, end_dim=-1)
   (lazyLin): Linear(in_features=343, out_features=10, bias=True)
 ),
 tensor([-0.0134, -0.0387, -0.0594,  0.0336,  0.0421,  0.0608, -0.1380, -0.0417,
          0.1144,  0.0008], grad_fn=<ViewBackward0>))

## GP Code
Now we need to decide the genetic stuff.
1. Initial Population
2. Fitness Measure
3. Selection
4. Genetic Operators
    - Cloning or Crossover
    - Mutation
    
### Hyperparameter-landscape is vast. Here's a list:
- Net architecture
    - kind of layers, number of layers
        - for convolution/pooling: kernel size, stride, padding, (dilation) **[implemented]**
    - number of neurons per layer
    - activation function for each layer
- cost function
    - base term (e.g. square cost, log-likelihood, cross-entropy, ect.)
    - toppings 
        - regularization of weights (L2, L1, dropout, etc.)
- weights and biases optimization technique (= optimizer)
    - SGD (= stochastic gradient descent)
    - Hessian technique, i.e. momentum-based descent
    - PyTorch's various other (e.g. *Adam* optimizer)
- learning parameters
    - η ... learning rate
        - constant, or epoch-dependent, or accuracy-dependent, or a mix
    - \# of epochs
        - constant, or early stopping
    - (`mini_batch_size` - this one might be canonical)

### Create Random Population

In [268]:
# Calculate the image size after a layer has been applied
# assume all operations to be x/y symmetric
def output_size(h_in, kernel_size, stride, padding):
    
    # Calculate output height and width
    h_out = (h_in + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    #w_out = (w_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    
    return h_out # = w_out

In [274]:
import random
def create_random_population(pop_size: int, 
                             max_2d_block_count: int = 3, 
                             max_kernel_size: int = 11,
                             name_prefix="nn",
                             print_summary: bool = True) -> list[NN_individual]:
    population = []
    for i in range(pop_size):
        genes_2d_block = []
        input_image_size = IMAGE_HEIGHT # = IMAGE_WIDTH (assumed)
        name=name_prefix+str(i)
        if print_summary: print(f"Individual '{name}' <-- ({input_image_size} x {input_image_size})")
        for j in range(random.randint(1, max_2d_block_count)):
            if print_summary: print(f"\tBlock {j}")
            conv_kernel_size=min(random.randint(1,min(input_image_size, max_kernel_size)), random.randint(1,min(input_image_size, max_kernel_size)))
            conv_stride=random.randint(1,conv_kernel_size)
            conv_padding=random.randint(0,conv_kernel_size//2) # PyTorch: "pad should be at most half of effective kernel size"
            # update input image size after convolutional layer
            input_image_size = output_size(input_image_size, conv_kernel_size, conv_stride, conv_padding)
            pool_kernel_size=min(random.randint(1,min(input_image_size, max_kernel_size)), random.randint(1,min(input_image_size, max_kernel_size)), random.randint(1,min(input_image_size, max_kernel_size)))
            pool_stride=max(random.randint(1,pool_kernel_size), random.randint(1,pool_kernel_size))
            pool_padding=random.randint(0,pool_kernel_size//2) # PyTorch: "pad should be at most half of effective kernel size"
            block = Gene_2d_block(
                # the following lower/upper bound values are not fine-tuned
                out_channels=random.randint(3,15),
                conv_kernel_size=conv_kernel_size,
                conv_padding=conv_padding,
                conv_stride=conv_stride,
                pool_kernel_size=pool_kernel_size,
                pool_padding=pool_padding,
                pool_stride=pool_stride
            )
            genes_2d_block.append(block)
            input_image_size = output_size(input_image_size, pool_kernel_size, pool_stride, pool_padding)
            if print_summary: print(f"{block.toString(tab_count=2)} --> ({input_image_size} x {input_image_size})")
        population.append(NN_individual(genes_2d_block=genes_2d_block, name=name))
    return population
testPop = create_random_population(pop_size=7, max_2d_block_count=3, print_summary=False)
try:
    for ind in testPop:
        ind.eval()
        with torch.inference_mode():
            ind(testX)
except:
    print("oh, oh! exception")

### Fitness Evaluation
We want a population that:
- achieves high (validation/test data) accuracy after training
    - the final accuracy `acc(NN1(t_final))` of an individual `NN1` is used
- trains fast, i.e. takes little CPU time to achieve high accuracy called **Running Accuracy**
    - the individual's accuracy `acc(NN1(t))` is summed over given timestamps `t`, like `Σ_t{acc(NN1(t))}`
    - possibly we want to value early accuracy more, summing `Σ_t{acc(NN1(t))/t}` instead

In [270]:
def accuracy(ind: NN_individual, X_in, y_true):
    ind.eval()
    with torch.inference_mode():
        correct = torch.eq(ind(X_in), y_true).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(X_in)) * 100 
    return acc # in percent

# the momentary accuracy shall be the summand of the total running accuracy
def momentary_accuracy(t: float, ind: NN_individual, X_in, y_true):
    if t > 0:
        running_acc = accuracy(ind, X_in, y_true)/t
    else:
        print("Got time t <= 0 in calculation of running_accuracy!")
        Exception
    return running_acc

### Selection
To select, we need to train the population.

In [271]:
def train_model_one_epoch(ind: NN_individual, # <- model to be trained and then returned
                          train_dl,  # <- train dataloader
                          test_dl,   # <- test dataloader
                          optimizer, # <- e.g. torch.optim.SGD(ind.parameters(), lr=.1)
                          loss_fn=nn.CrossEntropyLoss(), 
                          device='cpu'):
  train_loss = 0
  for batch, (X, y) in enumerate(train_dl):
    ind.train()
    X, y = X.to(device), y.to(device)
    y_pred = ind(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Adjust train loss for number of batches
  train_loss /= len(train_dl)

  ### Testing loop
  test_loss_total = 0
  ind.eval()
  with torch.inference_mode():
    for batch, (X_test, y_test) in enumerate(test_dl):
      X_test, y_test = X_test.to(device), y_test.to(device)
      test_pred = ind(X_test)
      test_loss = loss_fn(test_pred, y_test)
      test_loss_total += test_loss

    test_loss_total /= len(test_dl)

  # Print out what's happening
  print(f"Individual {ind.name}'s loss: {train_loss:.3f}, test loss: {test_loss_total:.3f}")

We now train the whole population simultaneously for one epoch each.
The running accuracy is summed up on-the-fly.

In [272]:
import pandas as pd
def select(pop: list[NN_individual],
           train_dl,  # <- train dataloader
           test_dl,   # <- test dataloader
           optimizer, # <- e.g. torch.optim.SGD(ind.parameters(), lr=.1)
           max_epochs=5,
           loss_fn=nn.CrossEntropyLoss(), 
           device='cpu') -> list[NN_individual]:
  # each individual now obtains some fitness values that are now initialized:
  pop_fitness = pd.DataFrame([{"name": ind.name, "acc": 0, "running acc": 0} for ind in pop])
  # train each individual "simultaneously" by making the epoch-loop the outer one
  for epoch in range(max_epochs):
    print(f"*** Commencing epoch {epoch} / {max_epochs}. ***")
    for i in range(len(pop)):
      train_model_one_epoch(pop[i], 
                            train_dl=train_dl,
                            test_dl=test_dl,
                            optimizer=optimizer,
                            loss_fn=loss_fn,
                            device=device)
        

immediate TODOs:
- return `total_loss`, `total_acc` (total meaning over all batches) from `train_model_one_epoch`; probably in a dict
- use those values to fill the `pop_fitness` `DataFrame`

### Mutate